<a href="https://colab.research.google.com/github/Neilus03/DEEP-LEARNING-2023/blob/main/P5_Intro_to_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Convolutional Neural Networks

In this notebook we will see how to define and use a Convolutional Neural Network (CNN) for clasifying images.

Contrary to the MLPs we have used up to know, where each unit connects to all the units of the previous layer, a unit in a convoutional layer only "sees" a small region of the layer before, and applies a filter - a local operation - on this small region. The key idea is to apply this same filter to the whole image in a grid fashion, which results into a map of activations for this filter. Using many filters, results into a set of activation maps that when stacked together will form the output of the convolutional layer (each activation map corresponding to a "channel").

Typically, convolutions are followed by pooling layers that reduce the dimensionality of our intermedate result, and give us every time a little bit more of viewpoint invariance (at the cost of losing precise information about the location of things).

When dealing with images, our inputs, and all intermediate tensors we produce through convolution layers will be 3-dimensional tensors. And if we vectorise our operations and pass a whole batch of images in a single go, we will be dealing with 4-dimensional tensors. The ordering of the dimensions in pytorch is $(BatchSize \times nChannels \times Height \times Width)$.

After a series of convolutions and pooling operations, we typically want to flaten the images. The flattening operation collapses an array into a 1-dimensional vector. For example, if we have a tensor of $20 \times 5 \times 5$ $(C \times H \times W)$, its flattened version is a 1d vector of $500$ values. Now we can feed these $500$ values into a MLP for classifiying the image.

In [1]:
import torch #should be installed by default in any colab notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.numel()
    return np

In [2]:
# If this cell fails you need to change the runtime of your colab notebook to GPU
# Go to Runtime -> Change Runtime Type and select GPU
assert torch.cuda.is_available(), "GPU is not enabled"

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data

Before training we need data! For this notebook we will use the FashionMNIST dataset. Which is available in [torchvision](https://pytorch.org/vision/stable/index.html)


In [3]:
train_set = datasets.FashionMNIST("data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip()]))
val_set = datasets.FashionMNIST("data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=512, shuffle=False, num_workers=2)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



The images of FashionMNIST are of size $(28 \times 28)$ pixels, and have a single channel - the greyscale one. So they are given as a 2D tensor. You can easily check this:

In [4]:
img = train_set.data[0]
img.shape

torch.Size([28, 28])

## Filters and Convolutional layers

To define a CNN model, similarly to the MLP case, we need to define all the operations and then combine them into the forward pass. Apart from `nn.Linear()` layers, we will now use `nn.Conv2d()` layers, which correspond to 2D convolutions.

To perform a convolution, we need to define the size of our filters, and how many filters to use.

The size of the filter is given by its width and height (which will be equal), and the number of channels which has to be the same as the number of channels of the input it will be applied to. So, if `in_channels` is the number of channels of the input, and `kernel_size` is the width and height of our filter, then our filter will be a tensor of shape $(in\_channels \times kernel\_size \times kernel\_size)$.

The other piece of information needed is how many filters to use. Remember that each filter gives rise to one activation map, which in turn corresponds to one channel in the output. So the number of filters to use is the same as the number of output channels (`out_channels`) we want to have.

Internally, the set of weights that represent all the filters of a convolutional layer is stored as a single 4D tensor of shape (Number of filters, Number of channels, Height, Width), which would be equivalently $(out\_channels \times in\_channels \times kernel\_size \times kernel\_size)$

This is the minimum information needed to define a convolution layer. Unless otherwise specified, this would apply a default padding of zero and stride of 1. Have a look at the documentation for more details on the `nn.Conv2d()` layer.

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html?highlight=conv#torch.nn.Conv2d

Finally, remember the formula to calculate the size of the output $o$ given the input size $n$ the filter size $f$ the padding $p$ and the stride $s$:

$o = {\frac {(n + 2p - f)} s} + 1$

<br>
<span style="color:blue">

**Exercise 1**: Given the following convolutional layer:

`nn.Convd2(in_channels=20, out_channels=128, kernel_size=3)`

What is the shape of the weights and biases of the convolutional layer?

</span>

---

*Shape of Weights: (128, 3, 3, 20)*

*Shape of Biases: (128,)*

---

## Our Convolutional Model

Let's define now our first CNN model. 

In [6]:
class CNN(nn.Module):
    def __init__(self, n_feature, output_size):
        super(CNN, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size = 5)
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5)
        self.fc1 = nn.Linear(16 * n_feature, 50)
        self.fc2 = nn.Linear(50, 10)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        
    def forward(self, x): # 28 x 28 x 1
        x = self.conv1(x) #24 x 24 x n_features
        x = F.relu(x) #nothing changes
        x = self.maxpool(x) #12 x 12 x n_features
        x = self.conv2(x) # 8 x 8 x n_features
        x = F.relu(x) #nothing changes
        x = self.maxpool(x) #4 x 4 x n_features
        x = x.view(-1, self.n_feature) # 16* n_features (just a vector)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

<font color="green">**Exercise 2**: The model defined above has a shape mismatch problem. Calculate the correct input dimension for `fc1` and change its definition in the `__init__()` method and the call to `x.view()` in the forward pass. </font>

In [ ]:
# Your Code Here


## Hyperparameters

To use our CNN we need to create our model and define our hyperparameters. We will use an output size of 10, as there are 10 classes in FashionMNIST, and we will train with cross entropy loss and stochastic gradient descent.

In [ ]:
torch.backends.cudnn.benchmark = True
learning_rate = 1e-2
lambda_l2 = 1e-5
momentum = 0.5
torch.manual_seed(0) # seed for reproductibility

output_size = 10  # there are 10 classes

model = CNN(6, output_size)

print(f"Number of parameters {get_n_params(model)}:")

# move model to gpu if available
model.to(device)

# nn package also has different loss functions.
# we use cross entropy loss for our classification task
criterion = torch.nn.CrossEntropyLoss() # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss

# we use the optim package to apply
# stochastic gradient descent for our parameter updates
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=lambda_l2) # built-in L2

<br>
<span style="color:blue">

**Exercise 3**: What does the line `torch.backends.cudnn.benchmark = True` do? How will it affect our runtime if the size of the input images vary a lot?</font>

> Hint: Have a look at https://pytorch.org/docs/stable/backends.html

</span>

---

*Your Answer Here*

---

## Training and Validation

We should also define our training and validation loops. These are similar to previous notebooks.

In [ ]:
@torch.no_grad()  # prevent this function from computing gradients 
def validate(criterion, model, loader):

    val_loss = 0
    correct = 0

    model.eval()

    for data, target in loader:

        data, target = data.to(device), target.to(device)

        output = model(data)
        loss = criterion(output, target)
        val_loss += loss.item()                                                              
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(loader.dataset)
    accuracy = 100. * correct / len(loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(loader.dataset), accuracy))


    return val_loss


def train(epoch, criterion, model, optimizer, loader):
    
    total_loss = 0.0

    model.train()

    for batch_idx, (data, target) in enumerate(loader):
        
        optimizer.zero_grad()

        data, target = data.to(device), target.to(device)
        
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss.item()))


        total_loss += loss.item()  #.item() is very important here? Why?

    return total_loss / len(loader.dataset)

### Run Training

In [ ]:
losses = {"train": [], "val": []}
for epoch in range(10):

    train_loss = train(epoch, criterion, model, optimizer, train_loader)
    val_loss = validate(criterion, model, val_loader)
    losses["train"].append(train_loss)
    losses["val"].append(val_loss)
    
    plt.plot(losses["train"], label="training loss")
    plt.plot(losses["val"], label="validation loss")

    plt.legend()
    plt.pause(0.000001)
    plt.show()

It may also be useful to visualize some qualitative examples of classification 

In [ ]:
with torch.no_grad():

    data, target = next(iter(val_loader))
    data, target = data.to(device), target.to(device)

    output = model(data)
    predictions = np.argmax(output.cpu().numpy(), axis=1).tolist()
    true = target.cpu().numpy().tolist()
      
    plt.figure(figsize=(16, 6))
    
    print("Correctly predicted: {}%".format(100*sum(p == t for p, t in zip(predictions, true))/len(predictions)) )

    for i in range(10):
        plt.subplot(2, 5, i + 1)
        image = data[i,...].cpu().numpy().reshape((28,28))
        plt.imshow(image, cmap="gray")
        plt.axis('off')
        plt.title('Predicted as {}\n True label is {}'.format(val_set.classes[predictions[i]], val_set.classes[true[i]], ))

<br>
<span style="color:blue">
    
**Exercise 4**: Now repeat the same process, but flip the images horizontally before you run them through the model. Is your accuracy affected? Why?

> Hint: Check out the `numpy.flip()` function at https://pytorch.org/docs/stable/generated/torch.flip.html

</span>

In [ ]:
# Your Code Here


## Visualising Filters

One thing that would be interesting to visualise is the weights of the filters. To do so, we just need to access the weights of the convolutional layer of interest, and turn them into images. Here's an example for the first convolutional layer.

In [ ]:
weight = model.conv1.weight.data.cpu().numpy()

# Display the filters
fig, ax = plt.subplots(1, weight.shape[0], figsize=(5, 5))

for i in range(weight.shape[0]):
    ax[i].imshow(weight[i, 0], cmap='gray') # show the first (and only) channel weights
    ax[i].axes.get_xaxis().set_visible(False)        
    ax[i].axes.get_yaxis().set_visible(False)

## Visualising Activation Maps

Another thing would be to visualise the activations of a convolutional layer for a specific image when it goes through the model. To do so, we need to somehow capture the output of the convolutional layer during the forward pass. This can be done with the function `nn.Conv2d.register_forward_hook()`. This function accepts as argument a callback function that will be called by the model when the forward pass takes place.

See some more information about this function below:

In [ ]:
nn.Conv2d.register_forward_hook??

To try this out, we will defineby hand a toy model that does edge detection, as well as a couple of images to apply it on

In [ ]:
simpleEdgeDetector = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)

# Define a vertical edge detector filter
simpleEdgeDetector.weight.data[0, 0] = torch.tensor([  [1., 0., -1.],
                                                   [1., 0., -1.],
                                                   [1., 0., -1.]])

# Define an horizontal edge detector filter
simpleEdgeDetector.weight.data[1, 0] = torch.tensor([  [1., 1., 1.],
                                                   [0., 0., 0.],
                                                   [-1., -1., -1.]])

simpleEdgeDetector.bias.data = torch.zeros(2)

simpleEdgeDetector = simpleEdgeDetector.to(device)


fakeImageBatch = torch.zeros(2, 1, 26, 26) # create a batch of two black images of (10, 10)
fakeImageBatch[0,0,3,3] = 1 # Paint white pixel (3, 3)
fakeImageBatch[1,0,6,6] = 1 # Paint white pixel (6, 6)

# Show the images
fig, ax = plt.subplots(2, 1, figsize=(5, 5))
ax[0].imshow(fakeImageBatch[0,0], cmap='gray')
ax[1].imshow(fakeImageBatch[1,0], cmap='gray')

Let's also define some helper functions to plot a grid of activation maps given the output of a convolution layer.

In [ ]:
def showImage(img, ax):
    # convert the tensor to numpy
    out = img.numpy()
    # Bring to the 0-255 range
    out = out - out.min()
    out = out / out.max()
    out = out * 255
    out = out.astype('uint8')
    # Plot image
    ax.axes.get_xaxis().set_visible(False)        
    ax.axes.get_yaxis().set_visible(False)
    ax.imshow(out, cmap='gray')

def showConvMap(conv_map):
    # Create a grid of images
    h = conv_map.shape[0] # = number of images in the batch
    w = conv_map.shape[1] # = number of activation maps per image
    fig, ax = plt.subplots(h, w, figsize=(10, 10))

    # Plot activation maps
    for i in range(conv_map.shape[0]):
        for j in range(conv_map.shape[1]):
            showImage(conv_map[i][j], ax[i, j])
    
    fig.tight_layout()

Now we can define our hook function, which just calls `showConvMap()` to plot the output activation maps for all the images of our batch and all the filters of this layer.

In [ ]:
def hook_ShowOutput(module, input, output):
    print("Output shape:", output.shape)
    showConvMap(output.cpu())

fakeImageBatch = fakeImageBatch.to(device)

with torch.no_grad():
    handle = simpleEdgeDetector.register_forward_hook(hook_ShowOutput)
    embed = simpleEdgeDetector(fakeImageBatch)
    handle.remove()

<br>
<span style="color:blue">
    
**Exercise 5**: Visualize the feature (activation) maps of the first and last convolutional layers of our CNN model for 10 images of your validation set. Use the code provided before as the basis for your visualisation. What conclusions can you draw?
    
</span>

In [ ]:
# Your Code Here


---

*Your Answer Here*

---